In [1]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
pd.set_option('display.max_colwidth', 10)
pd.set_option('display.max_columns', None)

In [3]:
cdf_applicant = pd.read_csv('data/cdf_applicant.csv', index_col=['ApplicantID'])
cdf_applicant_experience = pd.read_csv('data/cdf_applicant_experience.csv', index_col=['ApplicantID'])
cdf_stage = pd.read_csv('data/cdf_stage.csv', index_col=['StageID'])
cdf_pipeline = pd.read_csv('data/cdf_pipeline.csv', index_col=['PipelineID'])

In [4]:
df_merged = pd.merge(
    cdf_applicant, cdf_applicant_experience, on=['ApplicantID']
)

df_merged = pd.merge(
    df_merged, cdf_pipeline, on=['ApplicantID']
)

df_merged = pd.merge(
    df_merged, cdf_stage, on=['StageID']
)

df_merged.drop(columns=['StageID'], inplace=True)

df_merged.set_index(['ApplicantID'], inplace=True)

# drop testing input
# df_merged.drop(index=13, inplace=True)

In [5]:
df_merged.head(2)

,DiseaseHistory,Age,DriverLicenseType,Gender,IsUsingGlasses,MaritalStatus,Nationality,Strengthness,Weaknesses,TypeOfVehicle,Height,ExpectedSalary,CityID,ProvinceID,Industry,CompanyName,JobDescription,Position,Salary,YearsOfExperience,JobID,Label
ApplicantID,,,,,,,,,,,,,,,,,,,,,,
3,0,29,c,male,false,single,indonesia,pekerj...,pelupa...,motor,178,4000000,1,1,teleko...,pt ico...,mainte...,sales ...,3666666,1,6,Rejected
10,0,29,c,male,false,single,indonesia,konsis...,terlal...,motor,168,3700000,1,1,event ...,victor...,bertug...,market...,3200000,2,1,Rejected


In [6]:
'''Hal yang terlewat'''

df_merged.JobDescription.fillna('', inplace=True)

In [7]:
df_merged.isna().sum()

DiseaseHistory       0
Age                  0
DriverLicenseType    0
Gender               0
IsUsingGlasses       0
MaritalStatus        0
Nationality          0
Strengthness         0
Weaknesses           0
TypeOfVehicle        0
Height               0
ExpectedSalary       0
CityID               0
ProvinceID           0
Industry             6
CompanyName          0
JobDescription       0
Position             0
Salary               0
YearsOfExperience    0
JobID                0
Label                0
dtype: int64

In [8]:
# tfidf = TfidfVectorizer()
# mtrix = tfidf.fit_transform(df_merged.JobDescription)

# cosim = linear_kernel(mtrix, mtrix)

In [9]:
df_merged.head(2)

,DiseaseHistory,Age,DriverLicenseType,Gender,IsUsingGlasses,MaritalStatus,Nationality,Strengthness,Weaknesses,TypeOfVehicle,Height,ExpectedSalary,CityID,ProvinceID,Industry,CompanyName,JobDescription,Position,Salary,YearsOfExperience,JobID,Label
ApplicantID,,,,,,,,,,,,,,,,,,,,,,
3,0,29,c,male,false,single,indonesia,pekerj...,pelupa...,motor,178,4000000,1,1,teleko...,pt ico...,mainte...,sales ...,3666666,1,6,Rejected
10,0,29,c,male,false,single,indonesia,konsis...,terlal...,motor,168,3700000,1,1,event ...,victor...,bertug...,market...,3200000,2,1,Rejected


In [10]:
cat = ['DriverLicenseType', 'Gender', 'IsUsingGlasses', 'MaritalStatus', 'Nationality', 'TypeOfVehicle', 'CityID', 'ProvinceID', 'JobID', 'Label']
num = ['Age', 'Height', 'ExpectedSalary', 'Salary', 'YearsOfExperience']
txt = ['DiseaseHistory', 'Strengthness', 'Weaknesses', 'Industry', 'CompanyName', 'JobDescription', 'Position']

for col in cat:
    df_merged[col] = df_merged[col].astype('category').cat.codes

In [90]:
df_merged

,DiseaseHistory,Age,DriverLicenseType,Gender,IsUsingGlasses,MaritalStatus,Nationality,Strengthness,Weaknesses,TypeOfVehicle,Height,ExpectedSalary,CityID,ProvinceID,Industry,CompanyName,JobDescription,Position,Salary,YearsOfExperience,JobID,Label
ApplicantID,,,,,,,,,,,,,,,,,,,,,,
3,0,29,4,2,0,2,2,pekerj...,pelupa...,2,178,4000000,1,1,teleko...,pt ico...,mainte...,sales ...,3666666,1,5,1
10,0,29,4,2,0,2,2,konsis...,terlal...,2,168,3700000,1,1,event ...,victor...,bertug...,market...,3200000,2,0,1
13,0,0,4,2,0,2,2,swot,swot,2,180,0,1,1,rudi,mr,,it,0,0,166,1
18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,pemeri...,badan ...,membua...,staff ...,900000,3,1,1
21,0,29,4,2,1,2,2,0,-,2,162,4500000,1,1,it it,pt gae...,scope ...,java w...,3300000,3,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39997,0,57,4,2,1,1,2,instal...,menggu...,2,170,4500000,1,1,instal...,pt fuj...,mengaw...,superv...,6300000,3,546,0
40043,0,31,4,2,0,1,2,memili...,menyuk...,2,170,6000000,0,1,erp pr...,pt gbs...,saya b...,progra...,5500000,6,529,0
43028,0,48,1,2,0,1,2,0,0,0,0,0,40,3,outsou...,pt sum...,sebaga...,driver,0,2,589,0


In [219]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report

df = df_merged[num + cat + ['JobDescription']]

X = df.drop(columns=['Label'])
y = df['Label']

# print(X.shape, y.shape)

vectorizer = TfidfVectorizer()
text_train = vectorizer.fit_transform(X['JobDescription'])

# print(text_train.shape)

text_train = pd.DataFrame(text_train.todense(), columns=vectorizer.get_feature_names_out(), index=X.index)

X = X.drop(columns=['JobDescription'])

scaler = MinMaxScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns, index=X.index)

# print(X.shape)

X = pd.concat([X, text_train], axis=1)

# print(X.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42)

model = RandomForestClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(classification_report(y_pred, y_test))

confusion_matrix(y_pred, y_test)

              precision    recall  f1-score   support

           0       0.09      0.36      0.14        28
           1       0.99      0.95      0.97      2069

    accuracy                           0.94      2097
   macro avg       0.54      0.65      0.56      2097
weighted avg       0.98      0.94      0.96      2097



array([[  10,   18],
       [ 101, 1968]], dtype=int64)

In [220]:
df.Label.value_counts()

1    6637
0     351
Name: Label, dtype: int64

In [188]:
X.shape, y.shape

((21368, 1971), (6988,))

In [ ]:
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.svm import SVC
# from sklearn.ensemble import RandomForestClassifier



# # pipeline for text data
# text_features = 'JobDescription'
# text_transformer = Pipeline(steps=[
#     ('vectorizer', TfidfVectorizer())
# ])

# # pipeline for categorical data
# categorical_features = cat
# categorical_transformer = Pipeline(steps=[
#     ('onehot', OneHotEncoder(handle_unknown='ignore'))
# ])

# # you can add other transformations for other data types

# # combine preprocessing with ColumnTransformer
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('text', text_transformer, text_features),
#         ('cat', categorical_transformer, categorical_features)
# ])

# # add model to be part of pipeline
# clf_pipe =  Pipeline(steps=[('preprocessor', preprocessor),
#                    ("model", RandomForestClassifier())
# ])

# # # you can just use preprocessor by itself
# X_train = preprocessor.fit_transform(X_train)
# X_test = preprocessor.transform(X_test)
# clf_s= SVC().fit(X_train, y_train)
# clf_s.score(X_test, y_test)

# # # or better, you can use the whole.
# # clf_pipe.fit(X_train, y_train) 
# # clf_pipe.score(X_test, y_test)

In [69]:
# from sklearn.model_selection import train_test_split



# df = df_merged[cat + [txt[-2]]]

# X = df.drop(columns=['Label'])
# y = df['Label'].values.reshape((-1, 1))


# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=42)

In [110]:
# pipeline

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('numeric_features',
                                                 Pipeline(steps=[('selector',
                                                                  FunctionTransformer(func=<function get_numeric_data at 0x000001E5975AD1B0>))])),
                                                ('text_features',
                                                 Pipeline(steps=[('selector',
                                                                  FunctionTransformer(func=<function get_text_data at 0x000001E59A1A1480>)),
                                                                 ('vec',
                                                                  TfidfVectorizer())]))])),
                ('clf', RandomForestClassifier())])

In [ ]:
# import numpy as np
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.pipeline import Pipeline, FeatureUnion
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.preprocessing import FunctionTransformer
# from sklearn.model_selection import GridSearchCV, StratifiedKFold

# # Create Function Transformer to use Feature Union
# def get_numeric_data(x):
#     return [record[:-2].astype(float) for record in x]

# def get_text_data(x):
#     return [record[-1] for record in x]

# transfomer_numeric = FunctionTransformer(get_numeric_data)
# transformer_text = FunctionTransformer(get_text_data)

# # Create a pipeline to concatenate Tfidf Vector and Numeric data
# # Use RandomForestClassifier as an example
# pipeline = Pipeline([
#     ('features', FeatureUnion([
#             ('numeric_features', Pipeline([
#                 ('selector', transfomer_numeric)
#             ])),
#              ('text_features', Pipeline([
#                 ('selector', transformer_text),
#                 ('vec', TfidfVectorizer())
#             ]))
#          ])),
#     ('clf', RandomForestClassifier())
# ])

# # Grid Search Parameters for RandomForest
# param_grid = {'clf__n_estimators': np.linspace(1, 100, 10, dtype=int),
#               'clf__min_samples_split': [3, 10],
#               'clf__min_samples_leaf': [3],
#               'clf__max_features': [7],
#               'clf__max_depth': [None],
#               'clf__criterion': ['gini'],
#               'clf__bootstrap': [False]}

# param_grid = {
#     'clf__n_estimators': np.linspace(1, 100, 2, dtype=int)
# }

# # Training config
# kfold = StratifiedKFold(n_splits=7)
# scoring = {'Accuracy': 'accuracy', 'F1': 'f1_macro'}
# refit = 'F1'

# # Perform GridSearch
# rf_model = GridSearchCV(pipeline, param_grid=param_grid, cv=kfold, scoring=scoring, 
#                          refit=refit, n_jobs=-1, return_train_score=True, verbose=1)
# rf_model.fit(X_train, Y_train)
# rf_best = rf_model.best_estimator_

In [ ]:
# pipeline.n_features_in_